<a href="https://colab.research.google.com/github/knobs-dials/wetsuite-dev/blob/main/notebooks/extras/extras_diagnose_kamervragen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# (only) in colab, run this first to install wetsuite from (the most recent) source.   For your own setup, see wetsuite's install guidelines.
!pip3 install -U --no-cache-dir --quiet https://github.com/scarfboy/wetsuite-dev/archive/refs/heads/main.zip

# Purpose of this notebook

Inspect how many cases of question-asnwer mismatch are there in the `kamervragen` dataset.

In [2]:
import wetsuite.datasets

kamervragen = wetsuite.datasets.load('kamervragen-struc')

In [3]:
print(kamervragen.description)


    Questions from members of the parliament (at the moment only tweede kamer) to the government.

    Source: The KOOP SRU repositories,  specifically the ah-tk documents (aanhangsels, tweede kamer) within the officielepublicaties set.
    Restricted to documents that contain answers (ignoring documents that are only the questions, or only a note that they will be answered later)

    The structure of .data looks something like:
       {'ah-tk-20102011-2857': {
            'identifier': 'ah-tk-20102011-2857',
            'urls':   [ 'https://repository.overheid.nl/frbr/officielepublicaties/ah-tk/20102011/ah-tk-20102011-2857/1/metadata/metadata.xml',
                        'https://repository.overheid.nl/frbr/officielepublicaties/ah-tk/20102011/ah-tk-20102011-2857/1/xml/ah-tk-20102011-2857.xml',
                        ... 
                      ],
            'issued':       '2011-06-17',
            'available':    '2011-06-23',
            'vergaderjaar': '2010-2011',
            

# How many cases are there of grouped answers leading to mismatches in numbering?

In [4]:
mismatched_qa, matched_qa = 0,0

for id, details in list(kamervragen.data.items()):
    vraagdata = details['vraagdata']

    vraag_numbers = set()
    antwoord_numbers = set()

    for qa_number in vraagdata:
        if 'vraag' in vraagdata[qa_number]:
            vraag_numbers.add( qa_number )
        if 'antwoord' in vraagdata[qa_number]:
            antwoord_numbers.add( qa_number )

        diff = vraag_numbers.symmetric_difference( antwoord_numbers )
        if len(diff) == 0:
            matched_qa    += 1
        else:
            mismatched_qa += 1
            if 0:  # showing all cases in the dataset would be very spammy
                print('\n=== %s ==='%id)
                print("Vragen:     %s"%sorted(vraag_numbers))
                print("Antwoorden: %s"%sorted(antwoord_numbers))
                print("Difference: %s"%sorted(diff))
                print( details['urls'][-1] )  # mention the document we used, so we can look at it

print( f"Matched: {matched_qa}     Not matched: {mismatched_qa}" )

Matched: 219302     Not matched: 12259
